In [1]:
import ssid
import quakeio
from ssid import modes
from utilities import Config, extract_channels, list_files, print_modes, mode_statistics

### Data inputs

In [2]:
directory = "uploads/painter"
pattern  = "*.[zZ][iI][pP]"

In [3]:
data_conf = Config()
data_conf.d = 8 # decimation factor

# PAINTER RIO DELL TRANSVERSE (CASE 1)
data_conf.inputs  = [17,3,20]
data_conf.outputs = [9,7,4]
# # PAINTER RIO DELL LONGITUDINAL (CASE 2)
# data_conf.inputs  = [15,1,18]
# data_conf.outputs = [11]

# # HWY8/MELOLAND TRANSVERSE (CASE 1)
# data_conf.inputs  = [2],
# data_conf.outputs = [5,7,9]
# # HWY8/MELOLAND TRANSVERSE (CASE 2)
# data_conf.inputs  = [11,2,26],
# data_conf.outputs = [5,7,9]
# # HWY8/MELOLAND LONGITUDINAL (CASE 3)
# data_conf.inputs  = [12,4,25],
# data_conf.outputs = [27,8]

# # CROWLEY TRANSVERSE (CASE 1)
# data_conf.inputs = [4]
# data_conf.outputs = [6,7,9]
# # CROWLEY TRANSVERSE (CASE 2)
# data_conf.inputs = [6,4,9]
# data_conf.outputs = [7]
# # CROWLEY TRANSVERSE (CASE 3)
# data_conf.inputs = [4]
# data_conf.outputs = [7]
# # CROWLEY LONGITUDINAL (CASE 4)
# data_conf.inputs = [5]
# data_conf.outputs = [8]

# # RIDGECREST TRANSVERSE (CASE 1)
# data_conf.inputs  = [4]
# data_conf.outputs = [6,7,9]

# # CAPISTRANO TRANSVERSE (CASE 1)
# data_conf.inputs  = [4]
# data_conf.outputs = [10,7]

# # HAYWARD TRANSVERSE (CASE 1)
# data_conf.inputs  = [2,7,25,18]
# data_conf.outputs = [13,15,23,20]
# # HAYWARD LONGITUDINAL
# data_conf.inputs  = [3,6,17],
# data_conf.outputs = [12,14,19]

# # BERNARDINO TRANSVERSE BENT 3 (CASE 1)
# data_conf.inputs  = [6]
# data_conf.outputs = [7,8]
# # BERNARDINO TRANSVERSE BENT 8 (CASE 2)
# data_conf.inputs  = [24]
# data_conf.outputs = [19,20]
# # BERNARDINO LONGITUDINAL BENT 3 (CASE 3)
# data_conf.inputs  = [4]
# data_conf.outputs = [10]
# # BERNARDINO LONGITUDINAL BENT 8 (CASE 4)
# data_conf.inputs  = [22]
# data_conf.outputs = [17,18]


### Method Inputs

### General Parameters
parameter | value
----------|-------
`p ` | number of output channels
`q ` | number of input channels
`nt` | number of timesteps
`dt` | timestep
`d`  | decimation (downsampling) factor

### Specific to Observer Kalman Identification (OKID)
parameter | value
----------|-------
`m`  | number of Markov parameters to compute (at most = nt)

### Specific to Eigensystem Realization Algorithm (ERA)
parameter | value
----------|-------
`no` | number of observability parameters
`nc` | number of controllability parameters
`r ` | reduced model order

### Specific to Data Correlations (DC)
parameter | value
----------|-------
`a`  | (alpha) number of additional block rows in Hankel matrix of correlation matrices
`b`  | (beta) number of additional block columns in Hankel matrix of correlation matrices
`l`  | initial lag
`g`  | lag (gap) between correlations

### Specific to System Realization with Information Matrix (SRIM)
parameter | value
----------|-------
`no` | number of steps used for identification (prediction horizon)

In [4]:
# Set Parameters
conf = Config()
conf.m  = 400
conf.no = 50
conf.nc = 50
conf.r  = 10
conf.a  = 0  
conf.b  = 0  
conf.l  = 10 
conf.g  = 3

In [5]:
files = list_files(directory, pattern)
event_modes = []
for file in files:
    print(file)

    try:
        event = quakeio.read(file, exclusions=["*filter*", "*date*"])
        print("peak acceleration (cm/s/s):", event["peak_accel"])
        inputs, dt = extract_channels(event, data_conf["inputs"], decimate=data_conf["d"])
        outpts, dt = extract_channels(event, data_conf["outputs"], decimate=data_conf["d"])
        realization = ssid.system(method="srim", input=inputs, output=outpts, **conf)
    except Exception as e:
        print(e)
        continue
    ss_modes = modes.modes(realization,dt)
    event_modes.append(list(ss_modes.values()))
    print_modes(ss_modes)

event_frequencies = mode_statistics(event_modes, "freq")
event_frequencies

uploads/painter/nc73201181_ce89324p.zip
peak acceleration (cm/s/s): 243.242
Spectral quantities:
         T(s)      ζ
      0.2658     0.1054   
      0.2641     0.003674 
      0.2432     0.00376  
      0.2289     0.01486  
Mean Period(s): 0.2504793131254741
Standard Dev(s): 0.015307510934595071
uploads/painter/nc73821036_ce89324p.zip
peak acceleration (cm/s/s): -1360.038
Spectral quantities:
         T(s)      ζ
      0.358      0.02692  
      0.3171     0.03952  
      0.2871     -0.01661 
      0.2766     0.1134   
      0.1779     0.1052   
Mean Period(s): 0.2833471835793703
Standard Dev(s): 0.05981093831381329
uploads/painter/RioDell_Petrolia_Processed_Data.zip
peak acceleration (cm/s/s): 30.272
Spectral quantities:
         T(s)      ζ
      0.2435     0.02224  
      0.2298     0.01618  
      0.2069     0.0246   
      0.188      0.006616 
      0.1653     0.00988  
Mean Period(s): 0.20670472502500825
Standard Dev(s): 0.02812064313428049


[{'freq': 4.112576952979431,
  'damp': 0.00375999881536396,
  'modeshape': array([0.00087506+0.05196234j, 0.02670332+0.12721886j,
         0.01050769+0.12349344j]),
  'cnd': 1.2898167916973897,
  'distance': -0.14071242625493027},
 {'freq': 3.6148510483366842,
  'damp': 0.11344683119901862,
  'modeshape': array([ 0.09619339+0.24187151j,  0.1141495 +0.24351288j,
         -0.03573611+0.15548571j]),
  'cnd': 1.1588715822812645,
  'distance': -0.6915422736363749},
 {'freq': 4.350942455849533,
  'damp': 0.016176990439676905,
  'modeshape': array([0.09339581-0.04797893j, 0.18150164-0.11840353j,
         0.08595544-0.06274352j]),
  'cnd': 1.2653873312414072,
  'distance': 0.12308504207491686}]

### Painter St Rio Dell Hwy 101 89324
![Painter St Rio Dell Hwy 101 89324 Sensors 1](./figures/89324_Page_1.png)
![Painter St Rio Dell Hwy 101 89324 Sensors 2](./figures/89324_Page_2.png)

### Meloland Hwy 8 El Centro 01336
![Meloland Hwy 8 El Centro 01336 Sensors](./figures/01336.png)

### Lake Crowley Hwy 395 54730
![Lake Crowley Hwy 395 54730 Sensors](./figures/54730.png)

### Ridgecrest 33472
![Ridgecrest 33472 Sensors](./figures/33742.png)

### Capistrano 13795
![Capistrano 13795 Sensors](./figures/13795.png)

### Hayward Hwy 580-238 58658
![Hayward Hwy 580-238 58658 Sensors 1](./figures/58658_Page_1.png)
![Hayward Hwy 580-238 58658 Sensors 2](./figures/58658_Page_2.png)

### San Bernardino I10-215 23631
![San Bernardino I10-215 Sensors](./figures/23631.png)